# Modélisation

Nous allons faire une régression log-log level :

In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
%run Fusion_donnees.ipynb

Exception: File `'Fusion_donnees.ipynb.py'` not found.

**Préparation de la base de données avec les variables d'intérêt en log**

In [ ]:
#création de la base avec les variables d'intérêt
base_reg = df_dummies[['Total', 'Population', 'PIB par hab', 'Investissement sport', 'Annee_1992', 'Annee_1996', 'Annee_2000', 'Annee_2004', 'Annee_2008', 'Annee_2012', 'Annee_2016', 'Annee_2020', 'Pays_hote_oui_non']]
base_reg

In [7]:
#transformation en log des variables
base_reg['log_Population'] = np.log(base_reg["Population"])
base_reg['log_Total'] = np.log(base_reg["Total"])
base_reg['log_PIB par hab'] = np.log(base_reg["PIB par hab"])
base_reg['log_Investissement Sport'] = np.log(base_reg["Investissement sport"])
base_reg

NameError: name 'base_reg' is not defined

**Création de la table de corrélation entre les variables**

In [ ]:
base_corr = base_reg[['log_Total', 'log_Population', 'log_PIB par hab', 'log_Investissement Sport', 'Pays_hote_oui_non']]

sns.heatmap(base_corr.corr(), cmap='coolwarm', annot=True, fmt=".2f")

**Régression linéaire**

In [ ]:
indices_to_keep = ~base_reg.isin([np.nan, np.inf, -np.inf]).any(1)

base_reg = base_reg[indices_to_keep].astype(np.float64)

In [ ]:
# création de Y :

y = base_reg[['log_Total']].values.ravel()

#création de X
x = base_reg.drop(["Total", "PIB par hab", 'Population', 'Investissement sport', 'log_Total', 'log_PIB par hab', 'log_Investissement Sport', 'Pays_hote_oui_non'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2
)

In [ ]:
ols = LinearRegression()
model = ols.fit(X_train, y_train)

print(ols.intercept_, ols.coef_)

In [ ]:
# prédictions sur la base de test
y_pred = model.predict(X_test)



In [ ]:

rmse = sklearn.metrics.mean_squared_error(y_test, y_pred, squared = False)

print('Mean squared error: %.2f'% rmse)